In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url,time_out=10)
    contest=data.json()
    return contest

In [23]:
# 获取高德地图API地址信息
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
        print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except ConnectionError as e: # HTTPError
            print("请求出错")
            pass
        else:
            if ('geocodes' in temp):
                re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    re_Dict_gaode["re_citycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [5]:
# 获取百度地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [6]:
# ak码
ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [7]:
# 导入数据
inputpath = "H:\\work\\basic\\province\\DEC18\\"
inputFileName = "浙江(所在省).xlsx"
data = pd.read_excel(inputpath + inputFileName)


In [8]:
data.columns 

Index(['Unnamed: 0', '订单编号', '订单状态', '订单子状态', '订单挂起原因', '安装失败原因', '订单来源', '店铺',
       '订单金额', '订单生成时间', '交易完成时间', '发货时间', '物流单回填时间', '入网用户姓名', '入网身份证号',
       '入网手机号', '收货人姓名', '收货地址', '收货人电话号码', '邮编', '所在省 / 市 / 县', '号码归属地',
       'CPS推荐人', '订单备注', '物流单号', '物流签收时间', '承运商', '大小卡类型', '写卡渠道', '销售品编号',
       '销售品名称', '销售品类型', '套餐', '主号码', '副号码', '现金预存款', '靓号低消', '靓号等级', '靓号预存款',
       '营业厅送货方式', '是否线下模式', '是否线下转线上', '线下转线上原因', '营业厅送货iccid', '所在省', '所在市',
       '所在县', '号码归属省', '号码归属市'],
      dtype='object')

In [9]:
data["承运商"].value_counts()

EMS        11285
京东物流        6556
韵达          5673
京东线下        3760
骏飞通信专营店        1
Name: 承运商, dtype: int64

In [10]:
df = data[["所在省","所在市","收货地址","订单编号","承运商"]]

In [11]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\py35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
data.shape

(38403, 49)

In [13]:
df.head()

,所在省,所在市,收货地址,订单编号,承运商,address_all
0,浙江,绍兴市,南明街道城南路56号,600104833000008318120495751796,EMS,浙江省绍兴市南明街道城南路56号
1,浙江,温州市,浙江温州市苍南县马站镇王孙村97号,690000000001008318120420508887,EMS,浙江省温州市浙江温州市苍南县马站镇王孙村97号
2,浙江,杭州市,长河街道中赢康康谷3栋134重庆烤鱼,690000000001008318120420508800,京东物流,浙江省杭州市长河街道中赢康康谷3栋134重庆烤鱼
3,浙江,台州市,杜桥镇半阳亚奇偏光,600404864000008318120495751308,韵达,浙江省台州市杜桥镇半阳亚奇偏光
4,浙江,杭州市,仓前街道昌源清苑7号楼四单元502,690000000001008318120420508753,京东物流,浙江省杭州市仓前街道昌源清苑7号楼四单元502


In [14]:
df_EMS = df[df["承运商"] == "EMS"]
df_JDWL = df[df["承运商"] == "京东物流"]
df_YD = df[df["承运商"] == "韵达"]
df_JDXX = df[df["承运商"] == "京东线下"]

In [24]:
list_JDXX_gaode = get_receiver_lnglat_gaode(df_JDXX["address_all"])

浙江省杭州市塘栖镇塘北村埂头圩42号
浙江省台州市石塘镇上马工业区中马传动
浙江省嘉兴市云东公寓北门
浙江省丽水市水阁工业区云景路105号
浙江省宁波市时代大道一号逐浪健身
浙江省金华市罗埠镇进步轮胎公司
浙江省金华市罗埠镇进步轮胎公司
浙江省宁波市咸祥镇球山五一村里六份91号
浙江省绍兴市孙端镇鼎天天强纺织品有限公司
浙江省金华市象珠镇派溪吕村雪兵手机店
浙江省金华市象珠镇派溪吕村雪兵手机店
浙江省宁波市桃源街道杏蒋村豪丰易成公司。
浙江省杭州市祥茂路2号影天印业3号楼二楼CCI（大门进来左手边第一栋楼）
浙江省台州市前洋头里9弄20号
浙江省杭州市大成名座2-3-502
浙江省台州市前洋头里9弄20号
浙江省温州市梧田街道，红梅路21号
浙江省杭州市千岛湖镇明珠一区自提柜
浙江省嘉兴市濮院镇东方新天地6栋10号【后门】
浙江省绍兴市汇商路宝兴花园
浙江省杭州市银湖街道杜墓村72号
浙江省绍兴市汇商路宝兴花园
浙江省湖州市凤凰街道，王家庄门号217
浙江省绍兴市浦口街道浦慧路8号百世快运
浙江省宁波市高桥镇古庵村何家住宅区
浙江省宁波市大运路151号华业钢结构
浙江省宁波市高桥镇古庵村何家住宅区
浙江省金华市歌山镇白泥坎村采卢红糖厂
浙江省绍兴市道墟镇长溇村
浙江省嘉兴市昌盛南路现代广场2号楼603室
浙江省杭州市市心南路175号房地产大厦9楼
浙江省嘉兴市浙江省嘉兴市桐乡市凤鸣经济开发区强业路62号
浙江省金华市苏溪镇徐樟塘村2幢三单元
浙江省杭州市滨文路548号
浙江省嘉兴市新塍镇富园新村东区28号
浙江省嘉兴市浙江省嘉兴市嘉善县惠民镇开发区路19号
浙江省宁波市浙江省宁波市北仑区春晓镇五鹿山路199号世茂海滨花园东区
浙江省杭州市杭州市萧山区瓜沥镇瓜港一路环华制品包装有限公司
浙江省杭州市西兴街道风情苑11幢1单元自提柜
浙江省金华市西山路1111号溪西帝景
浙江省金华市白龙桥镇怡虹路九号
浙江省嘉兴市中国服装城A区3楼南C08，润泽羽绒
浙江省宁波市丰泽路246号德佑
浙江省温州市江滨西路华中大厦3幢
浙江省金华市江东街道山口小区153幢2单元店面
浙江省温州市旭阳路新体育馆王磊石材
浙江省宁波市馨兴公寓
浙江省嘉兴市黄湾镇尖山新区新民苑7/2/108
浙江省杭州市义蓬镇义蓬购物中心三楼小涮锅
浙江省舟山市浙江省舟山市定海区临城尚东城A区东门岗
浙江省杭州

ConnectionError: HTTPConnectionPool(host='restapi.amap.com', port=80): Max retries exceeded with url: /v3/geocode/geo?key=c71d9eda293d20db64955275557d92d4&address=%E6%B5%99%E6%B1%9F%E7%9C%81%E5%AE%81%E6%B3%A2%E5%B8%82%E8%A5%BF%E5%B7%A5%E4%B8%9A%E5%9B%AD%E5%8C%BA%E4%B8%83%E5%8D%81%E4%BA%8C%E5%8F%B7%E7%AB%8B%E8%BF%9C%E6%B1%BD%E9%85%8D%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D7CFB4B908>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [22]:
list_JDWL_gaode = get_receiver_lnglat_gaode(df_JDWL ["address_all"])

浙江省杭州市长河街道中赢康康谷3栋134重庆烤鱼
浙江省杭州市仓前街道昌源清苑7号楼四单元502
浙江省杭州市湖州街蔡马人家2-1-701
浙江省杭州市周家里20栋一单元202
浙江省杭州市东新街道颜三路万和玺园震轩美容美发
浙江省杭州市高尔夫路北中控技术富阳产业园海康微影
浙江省杭州市所前镇缪家村桥西166号
浙江省杭州市学林街杭州电子科技大学生活区4号楼
浙江省杭州市义蓬街道春意四路87号东尼造型
浙江省杭州市下沙经济开发区世贸江滨花园骏景湾9幢201
浙江省杭州市富春街道金秋大道518号富阳瑞丰老年公寓
浙江省杭州市山西省吕梁市孝义市新安街地区五交化小区门房收
浙江省杭州市良渚街道勾庄杭丝工业园|
浙江省嘉兴市独山港镇虎啸路山西沼39号
浙江省杭州市现代景苑1幢
浙江省杭州市三花工业园一号楼
浙江省杭州市凤起东路农业厅第三住宅区
浙江省杭州市长浜路718号与新天地街交叉口华盛达生活购物中心四楼大鼓米线
浙江省杭州市求是路299号
浙江省杭州市利尔达物联网科技园2幢302众商科创
浙江省杭州市丁桥镇阳光逸城20号楼2单元1403
浙江省嘉兴市当湖街道新华中路225号平湖大药店
浙江省杭州市下沙高教园东区学林街280号浙江经贸职业技术学院
浙江省杭州市浙江杭州市余杭区临平新洲路818号
浙江省杭州市杭海路263号招商银行之江支行
浙江省杭州市浙江杭州市萧山区春和钱塘6幢二单元1302
浙江省宁波市神马工业新区东南工二路。一车间
浙江省宁波市保税区大厦消防值班室
浙江省杭州市南堤路28号杭州江东花边都是有限公司
浙江省杭州市富阳新登镇永昌村永桥85号
浙江省杭州市杭州杭发发电设备有限公司
浙江省杭州市三墩镇古墩路圣苑小区16幢1单元2403
浙江省杭州市鹿山街道日月大道2号日月电器科宝1号楼
浙江省杭州市浙江杭州市富阳市银湖街道龙溪北路399号新尹美术学校
浙江省嘉兴市万业工业园依璐欣6D
浙江省金华市东阳市江北街道歌山北路280号
浙江省绍兴市大庆景苑11栋403
浙江省绍兴市四马路9号
浙江省杭州市三墩镇西园二路阿斯丽电器有限公司7号
浙江省杭州市明豪大厦26层今日头条
浙江省杭州市杭玻街120号丁信幼儿园
浙江省杭州市浙江杭州市余杭区方汇花苑13-1-601
浙江省杭州市宏达路乾北嘉苑速尔快递。
浙江省杭州市金地自在城东苑43栋1804
浙江省杭州市浙江杭州市

ConnectionError: HTTPConnectionPool(host='restapi.amap.com', port=80): Max retries exceeded with url: /v3/geocode/geo?key=c71d9eda293d20db64955275557d92d4&address=%E6%B5%99%E6%B1%9F%E7%9C%81%E6%9D%AD%E5%B7%9E%E5%B8%82%E6%B5%99%E6%B1%9F%E5%BB%BA%E8%AE%BE%E8%81%8C%E4%B8%9A%E6%8A%80%E6%9C%AF%E5%AD%A6%E9%99%A2zz (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D7CFBFBC88>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [ ]:
list_YD_gaode = get_receiver_lnglat_gaode(df_YD ["address_all"])

In [ ]:
list_EMS_gaode = get_receiver_lnglat_gaode(df_EMS["address_all"])

In [ ]:
df_EMS_gaode = pd.DataFrame(list_EMS_gaode)
df_JDXX_gaode = pd.DataFrame(list_JDXX_gaode)
df_JDWL_gaode = pd.DataFrame(list_JDWL_gaode)
df_YD_gaode = pd.DataFrame(list_YD_gaode )

In [ ]:
# 导出数据
outpath = "D:\\work\\basic\\province\\"
name_ANSI = "1812_info_alter_lnglat1.xlsx"
# name_UTF8 = "1812_info_alter_lnglat.csv"

In [ ]:
print("验证完整收货地址：")
print(add[0:3])
print("&%"*50)